### Load dependencies :

In [3]:
from warnings import filterwarnings
filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict,ShuffleSplit,GridSearchCV,KFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error , mean_absolute_error
from math import sqrt
import joblib
import json
import datetime

### Load and split data :

In [4]:
df=pd.read_csv('preprocessed_data_.csv')
df.head()

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,mprime_a_bin_2,mprime_a_bin_3,mprime_a_bin_4,mprime_g_bin_0,mprime_g_bin_1,mprime_g_bin_2,mprime_g_bin_3,mprime_g_bin_4,BOA_RT,alpha
0,0.938938,0.830099,-0.143116,0.205274,-0.748201,-0.102195,-0.148214,1.494961,-0.196407,-0.449844,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,429.332315,0.133511
1,0.072507,-1.551574,1.189851,0.880461,-0.191308,-0.932700,-0.652588,1.032032,1.082365,-0.470705,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,194.210750,0.363279
2,-2.603383,0.858439,-0.191497,0.141423,2.954771,-0.107965,-0.172161,-1.032376,-0.241683,0.598065,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,648.404343,0.280041
3,0.181490,0.807331,1.187661,-1.173034,-0.267672,-1.155707,-0.908316,0.025438,-1.062994,0.958397,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,885.138727,0.125099
4,-0.956225,0.171426,-0.715124,0.448112,0.653581,0.622586,0.469534,-0.757894,-0.363971,0.962627,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,602.697524,0.320636


In [5]:
X=df.drop(columns=['alpha'])
y=df['alpha']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [6]:
BOA_train=X_train['BOA_RT']
BOA_test=X_test['BOA_RT']
X_train.drop(columns=['BOA_RT'],inplace=True)
X_test.drop(columns=['BOA_RT'],inplace=True)

In [7]:
y.describe()

count    362503.000000
mean          0.288921
std           0.204526
min          -0.284378
25%           0.166093
50%           0.312022
75%           0.430461
max           0.853075
Name: alpha, dtype: float64

### Train the model with its best parameters :

In [8]:
from xgboost import XGBRegressor

model = XGBRegressor(
    learning_rate=0.1,
    n_estimators=300,
    max_depth=9
)


model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=9,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=300,
             n_jobs=None, num_parallel_tree=None, ...)

### Save model :

In [9]:
joblib.dump(model,'XGBoost.pkl')

['XGBoost.pkl']

### Evaluation :

In [12]:
from sklearn.metrics import mean_squared_error , mean_absolute_error

In [13]:
y_pred=model.predict(X_test)

In [14]:
y_test

159680   -0.224256
347342    0.376680
307069    0.238902
126659    0.471005
356896    0.128247
            ...   
326002    0.194530
190999   -0.142424
27703     0.149952
76624     0.143431
189606    0.229081
Name: alpha, Length: 90626, dtype: float64

In [15]:
MAE=mean_absolute_error(y_pred,y_test)
MAE

0.012707860855426146

In [16]:
X_train['BOA_RT']=BOA_train
X_test['BOA_RT']=BOA_test

In [17]:
X_test

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,mprime_a_bin_1,mprime_a_bin_2,mprime_a_bin_3,mprime_a_bin_4,mprime_g_bin_0,mprime_g_bin_1,mprime_g_bin_2,mprime_g_bin_3,mprime_g_bin_4,BOA_RT
159680,0.968688,0.690381,-1.610247,-0.842480,-0.765509,0.221345,-0.349359,-1.019351,2.036857,1.369018,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,914.444755
347342,-1.203300,0.908147,0.427997,-0.205544,0.900421,-0.673723,-0.603632,0.725515,-0.216203,-0.280011,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,594.575497
307069,-0.052629,-0.103307,1.356170,-2.028452,-0.101012,-1.229507,-0.940427,1.285479,-1.116649,0.798446,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,284.591943
126659,0.122052,-1.145261,-1.486535,-0.352838,-0.226280,0.457653,-0.026960,0.770059,2.211000,-0.862565,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,373.718733
356896,-0.314747,0.806773,-0.039354,-1.604842,0.097997,-0.740746,-0.841591,-1.634395,-0.617101,1.323553,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,832.137702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326002,0.570257,0.210242,0.225099,-0.963130,-0.524454,-0.741572,-0.750298,-1.102005,0.164011,0.682795,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,801.588284
190999,-0.122803,-1.748566,0.261369,1.172320,-0.049088,0.707174,0.919019,1.778795,0.595663,1.627898,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,32.089238
27703,0.718968,0.367986,0.122163,0.103691,-0.616840,-0.358504,-0.347800,-0.751896,-1.091658,0.908998,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,735.627029
76624,0.015910,0.770747,1.477093,-1.292452,-0.150825,-1.263014,-0.941725,-0.509560,-0.099483,0.205600,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,924.379368


In [18]:
cols_to_scale_inv = X_test.drop(columns=['mprime_a_bin_0', 'mprime_a_bin_1',
       'mprime_a_bin_2', 'mprime_a_bin_3', 'mprime_a_bin_4', 'mprime_g_bin_0',
       'mprime_g_bin_1', 'mprime_g_bin_2', 'mprime_g_bin_3', 'mprime_g_bin_4','BOA_RT'
       ]).columns
cols_to_encode_inv = ['mprime_a_bin_0', 'mprime_a_bin_1',
       'mprime_a_bin_2', 'mprime_a_bin_3', 'mprime_a_bin_4', 'mprime_g_bin_0',
       'mprime_g_bin_1', 'mprime_g_bin_2', 'mprime_g_bin_3', 'mprime_g_bin_4',
       ]

In [19]:
scaler=joblib.load('scaler.pkl')
encoder=joblib.load('encoder.pkl')
X_scaled_part = X_test[cols_to_scale_inv].values
X_encoded_part = X_test[cols_to_encode_inv].values
# 2. Inverser la standardisation
X_original_scaled = scaler.inverse_transform(X_scaled_part)

# 3. Inverser l'encodage
X_original_categoricals = encoder.inverse_transform(X_encoded_part)

In [20]:
df_reconstructed = pd.DataFrame(
    X_original_scaled,
    columns=['Tg_scat', 'Tg_abs', 'Ta_abs', 'SSA', 'GOD', 'AOD', 'AODS', 'SZA', 'Z',
       'R_scence', 'g1', 'Cos(SZA)', 'mu_g', 'mu_a', 'muprime_g', 'muprime_a',
       'mprime_g', 'mprime_a'],
    index=X_test.index
)
df_categoricals = pd.DataFrame(
    X_original_categoricals,
    columns=['mprime_a_bin', 'mprime_g_bin'],
    index=X_test.index
)
X_test_real = pd.concat([df_reconstructed, df_categoricals], axis=1)

In [23]:
X_test_real['BOA_RT']=X_test['BOA_RT']
X_test_real['Alpha_AI']=y_pred
X_test_real['alpha']=y_test

In [24]:
X_test_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,mprime_a_bin,mprime_g_bin,BOA_RT,alpha,Alpha_AI
159680,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,3187.429121,0.951776,0.951721,0.684362,0.152645,0,0,914.444755,-0.224256,-0.210449
347342,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,...,3186.056472,0.471695,0.470791,1.873413,1.217586,0,0,594.575497,0.376680,0.371614
307069,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,...,3185.507886,0.243492,0.241375,4.099723,4.110381,0,0,284.591943,0.238902,0.215276
126659,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,...,3187.535216,0.454355,0.453397,1.400187,0.288163,0,0,373.718733,0.471005,0.450295
356896,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,...,3185.812230,0.999159,0.999158,0.933753,0.732430,0,0,832.137702,0.128247,0.156564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326002,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,...,3186.288112,0.962308,0.962266,0.872219,0.472533,0,0,801.588284,0.194530,0.205964
190999,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,...,3186.551091,0.042399,0.030856,18.672395,11.328534,0,2,32.089238,-0.142424,-0.147039
27703,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,...,3185.523112,0.909141,0.909037,1.094304,1.074933,0,0,735.627029,0.149952,0.162459
76624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,...,3186.127582,0.859623,0.859456,1.011868,0.621184,0,0,924.379368,0.143431,0.128927


In [25]:
def calculate_BOA_IA(Tg_abs,Mg,Ta_abs,Ma,alpha,delta_g_scat,AODS):
    numerator= 1000*(Tg_abs**Mg)*(Ta_abs**Ma)
    denominator=1+alpha*delta_g_scat*Mg+(alpha*(1/3)*AODS)*Ma
    if denominator != 0 :
        BOA_ia= numerator / denominator
        return BOA_ia
    else :
        return 999

In [26]:
X_test_real['BOA_AI'] = X_test_real.apply(lambda row: calculate_BOA_IA(row['Tg_abs'], row['mprime_g'],row['Ta_abs'],row['mprime_a'],row['Alpha_AI'],row['GOD'],row['AODS']), axis=1)

In [28]:
X_test_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,muprime_g,muprime_a,mprime_g,mprime_a,mprime_a_bin,mprime_g_bin,BOA_RT,alpha,Alpha_AI,BOA_AI
159680,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,0.951776,0.951721,0.684362,0.152645,0,0,914.444755,-0.224256,-0.210449,914.244457
347342,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,...,0.471695,0.470791,1.873413,1.217586,0,0,594.575497,0.376680,0.371614,597.018080
307069,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,...,0.243492,0.241375,4.099723,4.110381,0,0,284.591943,0.238902,0.215276,290.135336
126659,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,...,0.454355,0.453397,1.400187,0.288163,0,0,373.718733,0.471005,0.450295,375.803676
356896,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,...,0.999159,0.999158,0.933753,0.732430,0,0,832.137702,0.128247,0.156564,825.363320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326002,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,...,0.962308,0.962266,0.872219,0.472533,0,0,801.588284,0.194530,0.205964,800.761836
190999,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,...,0.042399,0.030856,18.672395,11.328534,0,2,32.089238,-0.142424,-0.147039,-0.004774
27703,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,...,0.909141,0.909037,1.094304,1.074933,0,0,735.627029,0.149952,0.162459,734.434155
76624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,...,0.859623,0.859456,1.011868,0.621184,0,0,924.379368,0.143431,0.128927,927.366755


In [34]:
import math
def calculate_BOA_TOTAL(SZA,Z,alpha,tg_scat, tg_abs, optical_depth, albedo):   
    def muprime(z,h,µ):
        RAYON_TERRESTRE=6371
        eta = (RAYON_TERRESTRE*1000 + z) / h
        root = (eta*µ)**2  + 2 * eta + 1
        sum = (root)**0.5 - eta * µ
        if sum > 0 :
            return 1/sum
        return 1 
    E_TOA=1000
    Ha=2000
    Hg=9000
    angle_rad = math.radians(SZA)
    µ=math.cos(angle_rad)  
    Y_a=muprime(Z,Ha,µ)
    Y_g=muprime(Z,Hg,µ)
    Ma=math.exp(-Z/Ha)/Y_a
    Mg=math.exp(-Z/Hg)/Y_g
    delta_a_scat=optical_depth*albedo
    delta_g_scat=-math.log(tg_scat)
    Ta_abs=math.exp(-optical_depth*(1-albedo))
    numerator= E_TOA*(tg_abs**Mg)*(Ta_abs**Ma)
    denominator=1+alpha*delta_g_scat*Mg+(alpha*(1/3)*delta_a_scat)*Ma
    BOA_i= numerator / denominator
    return BOA_i

In [36]:
X_test_real['BOA_AI_2'] = X_test_real.apply(lambda row: calculate_BOA_TOTAL(row['SZA'], row['Z'],row['Alpha_AI'],row['Tg_scat'],row['Tg_abs'],row['AOD'],row['SSA']), axis=1)

In [37]:
X_test_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,muprime_a,mprime_g,mprime_a,mprime_a_bin,mprime_g_bin,BOA_RT,alpha,Alpha_AI,BOA_AI,BOA_AI_2
159680,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,0.951721,0.684362,0.152645,0,0,914.444755,-0.224256,-0.210449,914.244457,914.244457
347342,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,...,0.470791,1.873413,1.217586,0,0,594.575497,0.376680,0.371614,597.018080,597.018080
307069,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,...,0.241375,4.099723,4.110381,0,0,284.591943,0.238902,0.215276,290.135336,290.135336
126659,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,...,0.453397,1.400187,0.288163,0,0,373.718733,0.471005,0.450295,375.803676,375.803676
356896,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,...,0.999158,0.933753,0.732430,0,0,832.137702,0.128247,0.156564,825.363320,825.363320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326002,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,...,0.962266,0.872219,0.472533,0,0,801.588284,0.194530,0.205964,800.761836,800.761836
190999,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,...,0.030856,18.672395,11.328534,0,2,32.089238,-0.142424,-0.147039,-0.004774,-0.004774
27703,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,...,0.909037,1.094304,1.074933,0,0,735.627029,0.149952,0.162459,734.434155,734.434155
76624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,...,0.859456,1.011868,0.621184,0,0,924.379368,0.143431,0.128927,927.366755,927.366755


In [30]:
np.exp(-X_test_real['AOD']*(1-X_test_real['SSA']))

159680    0.715428
347342    0.902145
307069    0.987172
126659    0.726761
356896    0.859333
            ...   
326002    0.883558
190999    0.886881
27703     0.874129
76624     0.998250
189606    0.999232
Length: 90626, dtype: float64

In [31]:
X_test_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,muprime_g,muprime_a,mprime_g,mprime_a,mprime_a_bin,mprime_g_bin,BOA_RT,alpha,Alpha_AI,BOA_AI
159680,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,0.951776,0.951721,0.684362,0.152645,0,0,914.444755,-0.224256,-0.210449,914.244457
347342,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,...,0.471695,0.470791,1.873413,1.217586,0,0,594.575497,0.376680,0.371614,597.018080
307069,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,...,0.243492,0.241375,4.099723,4.110381,0,0,284.591943,0.238902,0.215276,290.135336
126659,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,...,0.454355,0.453397,1.400187,0.288163,0,0,373.718733,0.471005,0.450295,375.803676
356896,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,...,0.999159,0.999158,0.933753,0.732430,0,0,832.137702,0.128247,0.156564,825.363320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326002,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,...,0.962308,0.962266,0.872219,0.472533,0,0,801.588284,0.194530,0.205964,800.761836
190999,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,...,0.042399,0.030856,18.672395,11.328534,0,2,32.089238,-0.142424,-0.147039,-0.004774
27703,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,...,0.909141,0.909037,1.094304,1.074933,0,0,735.627029,0.149952,0.162459,734.434155
76624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,...,0.859623,0.859456,1.011868,0.621184,0,0,924.379368,0.143431,0.128927,927.366755


### MAE (E_BOA) :

In [32]:
BOA_pred=X_test_real['BOA_AI']
BOA_test=X_test_real['BOA_RT']
MAE=mean_absolute_error(BOA_pred,BOA_test)
MAE

6.455424310330715

In [14]:
from tensorflow.keras.models import load_model

model = load_model('deep_model1.keras')


2025-05-28 12:28:59.975425: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-28 12:28:59.980399: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-28 12:28:59.995332: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748428140.017956   59534 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748428140.025731   59534 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748428140.044708   59534 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [15]:
X_test

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,mprime_a_bin_1,mprime_a_bin_2,mprime_a_bin_3,mprime_a_bin_4,mprime_g_bin_0,mprime_g_bin_1,mprime_g_bin_2,mprime_g_bin_3,mprime_g_bin_4,BOA_RT
159680,0.968688,0.690381,-1.610247,-0.842480,-0.765509,0.221345,-0.349359,-1.019351,2.036857,1.369018,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,914.444755
347342,-1.203300,0.908147,0.427997,-0.205544,0.900421,-0.673723,-0.603632,0.725515,-0.216203,-0.280011,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,594.575497
307069,-0.052629,-0.103307,1.356170,-2.028452,-0.101012,-1.229507,-0.940427,1.285479,-1.116649,0.798446,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,284.591943
126659,0.122052,-1.145261,-1.486535,-0.352838,-0.226280,0.457653,-0.026960,0.770059,2.211000,-0.862565,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,373.718733
356896,-0.314747,0.806773,-0.039354,-1.604842,0.097997,-0.740746,-0.841591,-1.634395,-0.617101,1.323553,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,832.137702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326002,0.570257,0.210242,0.225099,-0.963130,-0.524454,-0.741572,-0.750298,-1.102005,0.164011,0.682795,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,801.588284
190999,-0.122803,-1.748566,0.261369,1.172320,-0.049088,0.707174,0.919019,1.778795,0.595663,1.627898,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,32.089238
27703,0.718968,0.367986,0.122163,0.103691,-0.616840,-0.358504,-0.347800,-0.751896,-1.091658,0.908998,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,735.627029
76624,0.015910,0.770747,1.477093,-1.292452,-0.150825,-1.263014,-0.941725,-0.509560,-0.099483,0.205600,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,924.379368


In [16]:
y_pred=model.predict(X_test.drop(columns=['BOA_RT']))

2833/2833 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step


In [17]:
X_test_real['alpha_deep']=y_pred

In [19]:
X_test_real['alpha']=y_test
X_test_real

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,mu_a,muprime_g,muprime_a,mprime_g,mprime_a,mprime_a_bin,mprime_g_bin,BOA_RT,alpha_deep,alpha
159680,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,3187.429121,0.951776,0.951721,0.684362,0.152645,0,0,914.444755,-0.216158,-0.224256
347342,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,...,3186.056472,0.471695,0.470791,1.873413,1.217586,0,0,594.575497,0.380552,0.376680
307069,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,...,3185.507886,0.243492,0.241375,4.099723,4.110381,0,0,284.591943,0.232299,0.238902
126659,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,...,3187.535216,0.454355,0.453397,1.400187,0.288163,0,0,373.718733,0.470436,0.471005
356896,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,...,3185.812230,0.999159,0.999158,0.933753,0.732430,0,0,832.137702,0.126723,0.128247
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
326002,0.909543,0.846892,0.883558,0.319999,0.094813,0.182056,0.058258,15.792506,1576.224940,0.696585,...,3186.288112,0.962308,0.962266,0.872219,0.472533,0,0,801.588284,0.193967,0.194530
190999,0.769135,0.464396,0.886881,0.823838,0.262489,0.681445,0.561401,88.522985,2102.182329,0.968884,...,3186.551091,0.042399,0.030856,18.672395,11.328534,0,2,32.089238,-0.148817,-0.142424
27703,0.939670,0.877695,0.874129,0.571705,0.062226,0.314101,0.179573,24.631558,46.223183,0.761757,...,3185.523112,0.909141,0.909037,1.094304,1.074933,0,0,735.627029,0.154897,0.149952
76624,0.797237,0.956342,0.998250,0.242298,0.226603,0.002312,0.000560,30.749744,1255.164902,0.559097,...,3186.127582,0.859623,0.859456,1.011868,0.621184,0,0,924.379368,0.140137,0.143431


In [21]:
def calculate_BOA_IA(Tg_abs,Mg,Ta_abs,Ma,alpha,delta_g_scat,AODS):
    numerator= 1000*(Tg_abs**Mg)*(Ta_abs**Ma)
    denominator=1+alpha*delta_g_scat*Mg+(alpha*(1/3)*AODS)*Ma
    
    BOA_ia= numerator / denominator
    return BOA_ia
    
       

In [24]:
X_test_real['BOA_deep'] = X_test_real.apply(lambda row: calculate_BOA_IA(row['Tg_abs'], row['mprime_g'],row['Ta_abs'],row['mprime_a'],row['alpha_deep'],row['GOD'],row['AODS']), axis=1)

In [25]:
X_test_real.head(10)

,Tg_scat,Tg_abs,Ta_abs,SSA,GOD,AOD,AODS,SZA,Z,R_scence,...,muprime_g,muprime_a,mprime_g,mprime_a,mprime_a_bin,mprime_g_bin,BOA_RT,alpha_deep,alpha,BOA_deep
159680,0.990261,0.940649,0.715428,0.348465,0.009787,0.513978,0.179103,17.879232,3858.242750,0.894296,...,0.951776,0.951721,0.684362,0.152645,0,0,914.444755,-0.216158,-0.224256,914.327265
347342,0.550236,0.983172,0.902145,0.498744,0.597408,0.205444,0.102464,61.931220,1112.943981,0.419185,...,0.471695,0.470791,1.873413,1.217586,0,0,594.575497,0.380552,0.376680,592.722235
307069,0.783352,0.785665,0.987172,0.068646,0.244174,0.013862,0.000952,76.068422,15.772549,0.729906,...,0.243492,0.241375,4.099723,4.110381,0,0,284.591943,0.232299,0.238902,286.119666
126659,0.818740,0.582204,0.726761,0.463992,0.199988,0.595434,0.276276,63.055806,4070.432067,0.251342,...,0.454355,0.453397,1.400187,0.288163,0,0,373.718733,0.470436,0.471005,373.775707
356896,0.730249,0.963376,0.859333,0.168593,0.314370,0.182341,0.030741,2.351437,624.459254,0.881197,...,0.999159,0.999158,0.933753,0.732430,0,0,832.137702,0.126723,0.128247,832.505466
288612,0.825671,0.564232,0.908966,0.677930,0.191559,0.296357,0.200909,89.409767,1485.490192,0.780309,...,0.032205,0.018693,26.326266,25.453480,1,3,39.535462,-0.148633,-0.148200,-0.008633
107719,0.717531,0.871646,0.801728,0.551692,0.331939,0.492933,0.271947,65.365335,549.760053,0.968267,...,0.418224,0.417142,2.249377,1.821116,0,0,367.013315,0.367694,0.370338,367.675822
246378,0.651030,0.574978,0.826722,0.267418,0.429199,0.259748,0.069461,36.518775,1063.488008,0.212873,...,0.803973,0.803731,1.105198,0.731065,0,0,360.230735,0.625577,0.631620,361.048725
76376,0.759927,0.879513,0.902394,0.274664,0.274533,0.141596,0.038891,1.716584,400.690622,0.885955,...,0.999552,0.999551,0.956884,0.818815,0,0,778.837549,0.159779,0.160782,779.041946
242293,0.848773,0.914715,0.628444,0.362730,0.163963,0.728903,0.264395,56.539442,712.576133,0.312174,...,0.552252,0.551561,1.672930,1.269617,0,0,402.838673,0.476463,0.480877,403.418765


In [105]:
BOA_pred=X_test_real['BOA_deep']
BOA_test=X_test_real['BOA_RT']
MAE=mean_absolute_error(BOA_pred,BOA_test)
MAE

3.5329352445173634